## Digital Image Processing, ECE419

Ernesto Colon

The Cooper Union Department of Electrical Engineering


February 19th, 2021

References:
- Digital Image Processing Using Matlab [DIPUM] by Gonzalez et al., 3rd ed
- Digital Image Processing by Gonzalez et al., 4th ed

**The following set of notes and example scripts are part of my independent study sessions and adapted from the references outlined above. My main reference is DIPUM and complemented with online documentation. As I go through the theory and textbook examples, I am translating the code and image processing techniques to Python.**

## Image Transformations Notes/Exercises/Sandbox

In [1]:
#Importing the libraries
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import ec_img_utils
import skimage.util
from skimage import io
import scipy.fft
from skimage import transform

#### Brief Notes on Inverse Transformation / Mapping

Starting with two spatial coordinate systems called the *input space*, $(w,z)$, and the *output space*, $(x,y)$, we can
define a geometric coordinate transformation that maps the input space to the output space:

\begin{equation}
(x,y) = T[(w,z)]
\end{equation}

where $T[\cdot]$ is called the *forward transformation* or *forward mapping*. If the forward mapping has an inverse, we
can express the output space in terms of the input space points as:

\begin{equation}
(w,z) = T^{-1}[(x,y)]
\end{equation}

where $T^{-1}[\cdot]$ is called the *inverse transformation* or *inverse mapping*.

#### Affine Transformations

An affine transformation is a mapping from one vector space to another, consisting of a linear part that is expressed
as a matrix multiplication and an additive part that is an *offset* or *translation*.

Following the notation used in DIPUM 3rd edition, an affine transformation can be expressed as:



\begin{bmatrix} x & y \end{bmatrix}
= \begin{bmatrix} w & z \end{bmatrix}
\begin{bmatrix}
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{bmatrix} + \begin{bmatrix} b_1 & b_2 \end{bmatrix}

We can re-write our affine transformation equation using homogeneous coordinates as follows:

\begin{bmatrix} x & y & 1 \end{bmatrix}
= \begin{bmatrix} w & z & 1 \end{bmatrix}
\begin{bmatrix}
a_{11} & a_{12} & 0\\
a_{21} & a_{22} & 0\\
b_1 & b_2 & 1
\end{bmatrix} =  \begin{bmatrix} w & z & 1 \end{bmatrix} $\textbf{T}$


where $\textbf{T}$ is called an *affine matrix* and the third column of $\textbf{T}$ is always
\begin{bmatrix} 0 & 0 & 1 \end{bmatrix}$^{t}$

#### Types of Affine Transformations

**Identity**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w$

$y = z$


**Scaling**

$\textbf{T}$ = \begin{bmatrix}
s_x & 0 & 0\\
0 & s_y & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = s_xw$

$y = s_yz$

**Rotation**

$\textbf{T}$ = \begin{bmatrix}
cos\theta & sin\theta & 0\\
-sin\theta & cos\theta & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w~cos\theta - z~sin\theta$

$y = w~sin\theta + z~cos\theta$

**Shear (horizontal)**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
\alpha & 1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w + \alpha~z$

$y = z$

**Shear (vertical)**

$\textbf{T}$ = \begin{bmatrix}
1 & \beta & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w$

$y = + \beta~w + z$

**Reflection**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
0 & -1 & 0\\
0 & 0 & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w$

$y = -z$


**Translation**

$\textbf{T}$ = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
\delta_x & \delta_y & 1
\end{bmatrix}

Corresponding coordinate equations

$x = w + \delta_x$

$y = z + \delta_y$

**Similarity Transformation**

An affine transformation that preserves angles between lines and that changes all distances in the same ratio is called
a *similarity transformation*. In other words, a similarity transformation preserves shapes.

Examples include: rotation, translation, and reflection transformations. Scaling is also a similarity transformation if
the horizontal and vertical scale factors are the same.

We can express the affine matrix for a similarity transformation as follows:

$\textbf{T}$ = \begin{bmatrix}
s~cos\theta & s~sin\theta & 0\\
-s~sin\theta & s~cos\theta & 0\\
b_1 & b_2 & 1
\end{bmatrix}

or

$\textbf{T}$ = \begin{bmatrix}
s~cos\theta & s~sin\theta & 0\\
s~sin\theta & -s~cos\theta & 0\\
b_1 & b_2 & 1
\end{bmatrix}

where the second form is needed to account for reflections.

Similarity transformations are useful in image processing applications involving solid and flat objects. As objects
or the imaging sensor moves, we can relate the motion with similarity transformations.

**Example 6.1 from DIPUM - Transforming points with an affine transformation**

**NOTE:** Python, or at least skimage, defines the affine matrix as the transpose of DIPUM's affine matrix definition.
Going back to the affine transformation equation outlined above, we see that the coordinate points are defined as row
vectors in DIPUM whereas scikit-image assumes column vectors.

In [2]:
# define affine matrix
affine_xfm_1 = np.array([[3., 0., 0.],[0., 2., 0.],[0., 0., 1.]])

# create similarity transform object with skimage
sim_xfm_1 = skimage.transform.SimilarityTransform(affine_xfm_1)

# confirm similarity transformation object has the correct affine matrix
print(sim_xfm_1.params)

[[3. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]]


In [3]:
# computing the forward transformation of a pair of points

# define input space coordinates (w, z) to transform, the input coords have shape P x 2
wz_pair_1 = np.array([[1, 1], [3., 2.]])
print("Input coords shape: ", wz_pair_1.shape, "\n")

# transform the input coords to the output coords
xy_pair_1 = skimage.transform.matrix_transform(wz_pair_1, sim_xfm_1.params)

# print results
print("Transformed output coords: \n", xy_pair_1)

Input coords shape:  (2, 2) 

Transformed output coords: 
 [[3. 2.]
 [9. 4.]]


In [4]:
# computing output space coordinates using the inverse mapping
wz_pair_2 = sim_xfm_1.inverse(xy_pair_1)

print("Input coords computed using the inverse mapping: \n", wz_pair_2)

Input coords computed using the inverse mapping: 
 [[1. 1.]
 [3. 2.]]


In [5]:
# modifying the affine transformation matrix
sim_xfm_1.params[0, 0] = 4

# confirm that the change took place
print("Modified affine matrix: \n", sim_xfm_1.params, "\n")

# apply transformation to our input coords
xy_pair_2 = skimage.transform.matrix_transform(wz_pair_1, sim_xfm_1.params)

# print results and confirm that our x-scaling took effect
print("Transformed output coords: \n", xy_pair_2)

Modified affine matrix: 
 [[4. 0. 0.]
 [0. 2. 0.]
 [0. 0. 1.]] 

Transformed output coords: 
 [[ 4.  2.]
 [12.  4.]]


Note how only the horizontal coordinates were scaled by a factor of 4 as expected.

The next example shifts the horizontal coordinates as a fraction of the vertical coordinates

In [6]:
# define affine matrix
affine_xfm_2 = np.array([[1, 0.4, 0], [0, 1, 0], [0, 0, 1]], dtype=np.float)

# define similarity transform object
sim_xfm_2 = skimage.transform.SimilarityTransform(matrix=affine_xfm_2)

# confirm
print("Similarity transform 2: \n", sim_xfm_2.params, "\n")

# apply transformation
xy_pair_3 = skimage.transform.matrix_transform(wz_pair_1, sim_xfm_2.params)

print("Transformed output coords: \n", xy_pair_3, "\n")

# going back with the inverse mapping
wz_pair_3 = sim_xfm_2.inverse(xy_pair_3)

print("Input coords computed using the inverse mapping: \n", wz_pair_3)

Similarity transform 2: 
 [[1.  0.4 0. ]
 [0.  1.  0. ]
 [0.  0.  1. ]] 

Transformed output coords: 
 [[1.4 1. ]
 [3.8 2. ]] 

Input coords computed using the inverse mapping: 
 [[1. 1.]
 [3. 2.]]


Note from the example above that only the horizontal coordinates where changed by the tranform.


In [7]:
from skimage import data

In [9]:
tform = transform.EuclideanTransform(rotation=np.pi / 12.,
                                     translation= (100, -20))
print(tform.params)

[[  0.96592583  -0.25881905 100.        ]
 [  0.25881905   0.96592583 -20.        ]
 [  0.           0.           1.        ]]


#### Projective Transformations (Homography)

A homography or projective transformation preserves lines but not necessarily parallelism. Projective transformations
are useful for reversing perspective distortion in an image. We define a projective transformation using a third auxiliary
coordinate, similar to the affine transformations. The auxiliary coordinate, however, is not a constant.


\begin{bmatrix} x' & y' & h \end{bmatrix}
= \begin{bmatrix} w & z & 1 \end{bmatrix}
\begin{bmatrix}
\alpha_{11} & \alpha_{12} & \alpha_{13}\\
\alpha_{21} & \alpha_{22} & \alpha_{23}\\
b_1 & b_2 & 1
\end{bmatrix}

were $\alpha_{13}$ and $\alpha_{23}$ are nonzero, and where $x' = hx$ and $y' = hy$.

**Creating a projective transformation with skimage**

In [19]:
# define transformation matrix
#proj_mat_1 = np.array([[1.8, -0.8, 0.2], [-0.1, 1.7, -0.2], [0.2, 0.4, 1.0]])
proj_mat_1 = np.array([[1.8, -0.8, 0.2], [-0.1, 1.7, -0.2], [0.2, 0.4, 1.0]]).T

# create projective trasnformation object
proj_xfm_1 = transform.ProjectiveTransform(matrix=proj_mat_1)

# input coordinate points
wz_pair_4 = np.array([[-1, -1], [1, -1], [1, 1], [-1, 1]])
#print(wz_pair_4)

# transform coordinate points
xy_pair_4 = transform.matrix_transform(wz_pair_4, proj_xfm_1.params)

# print transformed coordinate points
print("Transformed output coords: \n", xy_pair_4, "\n")

Transformed output coords: 
 [[-1.5        -0.5       ]
 [ 1.5        -1.5       ]
 [ 1.9         1.3       ]
 [-2.83333333  4.83333333]] 



#### Transforming Images

